Funcionamiento con las librerías `requests` y `BeautifulSoup`:

1. **Extraer lista de Pokemons y sus URLs desde la página principal:** [https://www.serebii.net/pokedex](https://www.serebii.net/pokedex) <br>
    1.1. Número, Nombre y URL de este paso pasarán a la tabla final

2. **Loop URL por URL de cada Pokémon y extraer las características**<br>
    2.1. Extraer tipo de Pokémon<br>
    2.2. Extraer los atributos Classification, Height, Weight, Capture rate<br>
    2.3. Extraer los Stats: HP, Attack, Defense, Special, Speed


In [77]:
import requests
from bs4 import BeautifulSoup
import time

url = r"https://www.serebii.net/pokedex"
page_data = requests.get(url)
soup = BeautifulSoup(page_data.content, 'html.parser')

base_url = "https://www.serebii.net"

select_element = soup.find('select', attrs={'name': 'SelectURL'})

# Crear listas para almacenar number, name y url
pokemon_numbers = []
pokemon_names = []
pokemon_urls = []

# Loop sobre los elementos de option dentro de select_element, comenzando desde la segunda opción
pokemon_list = []
for option in select_element.find_all('option')[1:]:
    # Extraer numero y nombre del Pokémon 
    number = option.get_text().split()[-2]  # Usar [-2] para corregir casos "RBY: 001 - 151"
    name = option.get_text().split()[-1]

    # Extraer the URL
    url = option['value']
    
    ## NÚMERO, NOMBRE, URL
    # Crear diccionario para el Pokémon en el loop
    pokemon_dict = {
        'Number': number,
        'Name': name,
        'URL': base_url + url
        }
    
    # Extraer características individuales de cada Pokémon
    poke_data = requests.get(base_url + url)
    poke_soup = BeautifulSoup(poke_data.content, 'html.parser')
    # Buscar elemento <td> con class "cen"
    td_cen = poke_soup.find('td', class_='cen')

    ## TIPO
    # Buscar elementos <img> en el elemento <td> para descrubrir el tipo de Pokémon, que en la web se muestra en forma de imagen
    img_elements = td_cen.find_all('img')
    types = []
    for img in img_elements:
        if 'src' in img.attrs:
            src = img['src']
            type_name = src.split('/')[-1].split('.')[0]
            types.append(type_name)

    # Añadir el Tipo al diccionario
    pokemon_dict['Type'] = types
    
    ## CLASSIFICATION, HEIGHT, WEIGHT, CAPTURE RATE
    # Buscar las líneas que contienen la información pertinente
    info_rows = poke_soup.find_all('tr')

    # Iterar sobre las líneas para extraer la información deseada 
        columns = row.find_all('td', class_='fooinfo')
        if len(columns) == 4:
            classification = columns[0].get_text()
            height_m = columns[1].get_text().split('\n')[1].strip().replace("m", "") # Obtener la altura en metros
            weight_kg = columns[2].get_text().split('\n')[1].strip().replace("kg", "") # Obtener el peso en kg
            capture_rate = columns[3].get_text()

    # Añadir los atributos al dicciionario
    pokemon_dict['Classification'] = classification
    pokemon_dict['Height_m'] = height_m
    pokemon_dict['Weight_kg'] = weight_kg
    pokemon_dict['Capture_rate'] = capture_rate
    
    
    ## STATS: HP, ATTACK, DEFENSE, SPECIAL, SPEED
    # Buscar los elementos <td> que son relevantes
    hp_element = poke_soup.find('td', class_='fooevo', text='HP')
    attack_element = poke_soup.find('td', class_='fooevo', text='Attack')
    defense_element = poke_soup.find('td', class_='fooevo', text='Defense')
    special_element = poke_soup.find('td', class_='fooevo', text='Special')
    speed_element = poke_soup.find('td', class_='fooevo', text='Speed')

    # Asegurar que al menos de los dos elementos existan
    if hp_element and attack_element:
        # Extraer valores de cada stat
        hp_value = hp_element.find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').get_text()
        attack_value = attack_element.find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').get_text()
        defense_value = defense_element.find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').get_text()
        special_value = special_element.find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').get_text()
        speed_value = speed_element.find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').find_next('td', class_='fooinfo').get_text()
    
    # Añadir al diccionario
    pokemon_dict['HP'] = hp_value
    pokemon_dict['Attack'] = attack_value
    pokemon_dict['Defense'] = defense_value
    pokemon_dict['Special'] = special_value
    pokemon_dict['Speed'] = speed_value
    
    # Añadir diccionario del Pokémon a la lista general
    pokemon_list.append(pokemon_dict)

In [81]:
# Crear pandas df a partir de la lista general
pokemon_dataset = pd.DataFrame(pokemon_list)

<IPython.core.display.Javascript object>

In [82]:
# Exportar a CSV
pokemon_dataset.to_csv('pokemon_dataset.csv', index=False)

In [83]:
pokemon_dataset

,Number,Name,URL,Type,Classification,Height_m,Weight_kg,Capture_rate,HP,Attack,Defense,Special,Speed
0,001,Bulbasaur,https://www.serebii.net/pokedex/001.shtml,"[grass, poison]",Seed Pokémon,0.7,6.9,45,45,49,49,65,45
1,002,Ivysaur,https://www.serebii.net/pokedex/002.shtml,"[grass, poison]",Seed Pokémon,1,13,45,60,62,63,80,60
2,003,Venusaur,https://www.serebii.net/pokedex/003.shtml,"[grass, poison]",Seed Pokémon,2,100,45,80,82,83,100,80
3,004,Charmander,https://www.serebii.net/pokedex/004.shtml,[fire],Lizard Pokémon,0.6,8.5,45,39,52,43,50,65
4,005,Charmeleon,https://www.serebii.net/pokedex/005.shtml,[fire],Flame Pokémon,1.1,19,45,58,64,58,65,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,147,Dratini,https://www.serebii.net/pokedex/147.shtml,[dragon],Dragon Pokémon,1.8,3.3,45,41,64,45,50,50
147,148,Dragonair,https://www.serebii.net/pokedex/148.shtml,[dragon],Dragon Pokémon,4,16.5,45 (RB)27 (Y),61,84,65,70,70
148,149,Dragonite,https://www.serebii.net/pokedex/149.shtml,"[dragon, flying]",Dragon Pokémon,2.2,210,45 (RB)9 (Y),91,134,95,100,80
149,150,Mewtwo,https://www.serebii.net/pokedex/150.shtml,[psychic],Genetic Pokémon,2,122,3,106,110,90,154,130
